In [1]:
import pandas as pd
import numpy as np
import glob
import math

import warnings
warnings.filterwarnings("ignore")

## Data Loading and Graph Constructing

In [2]:
CustInfo = pd.read_csv('../data/customerinformation.csv')
CustInfo['open_date'] = pd.to_datetime(CustInfo.Open_Date)

# CheqTrans = pd.read_csv('../../AMLAML/chequetransaction.csv')
# CheqTrans['trans_date'] = pd.to_datetime(CheqTrans.trandt)

# SARCase = pd.read_csv('../data/sarcase.csv')
# SARCase['created_date'] = pd.to_datetime(SARCase.Created_Date)

WireTrans = pd.read_csv('../data/new_wire.csv', index_col=0)
WireTrans['trans_date'] = pd.to_datetime(WireTrans.trandt)
# WireTrans = WireTrans.drop('Unnamed: 0', axis=1)
#WireTrans.head()

FileNotFoundError: [Errno 2] File b'../data/customerinformation.csv' does not exist: b'../data/customerinformation.csv'

In [16]:
# delete Non-IBW Customer_Segment_Code customerno
new_wire = WireTrans[WireTrans.customerno.isin(CustInfo[CustInfo.Customer_Segment_Code != 'IBW']['customerno'].values)]

# According Trf_Direction to build new feature  
new_wire['bnf_cust'] = new_wire.apply(lambda x: x['customerno'] if x['Trf_Direction'] == 'I' else np.nan ,axis=1)

new_wire['org_cust'] = new_wire.apply(lambda x: x['customerno'] if x['Trf_Direction'] == 'O' else np.nan ,axis=1)

In [18]:
class DateYM:
    def __init__(self, year, month):
        self.year = year
        self.month = month - 1    # 0 ~ 11, from Jan to Dec
        
    def export_tuple(self):
        return (self.year, self.month+1)
    
    def add_year(self, y):
        self.year += y
        
    def substract_year(self, y):
        self.year -= y
        
    def add_month(self, m):
        self.month += m
        self.year += math.floor(self.month / 12)
        self.month = self.month % 12
        
    def subtract_month(self, m):
        self.month -= m
        tmp_year = math.floor(self.month / 12)
        self.year += tmp_year
        self.month += -tmp_year * 12
        
    def is_larger_than(self, ym):
        return self.year*12 + self.month > ym.year*12 + ym.month
    
    def is_smaller_than(slef, ym):
        return self.year*12 + self.month < ym.year*12 + ym.month
    
    def is_equal(self, ym):
        return self.year*12 + self.month == ym.year*12 + ym.month

    
def list_date_tuples(from_date, to_date):
    ret = []
    tmp_date = DateYM(*from_date.export_tuple())
    while not tmp_date.is_larger_than(to_date):
        ret.append(tmp_date.export_tuple())
        tmp_date.add_month(1)
    return ret


def fetch_data_by_month(date_ym, trans_offset=6):
    year, month = date_ym
    
    from_date = pd.to_datetime("{}/{}/{}".format(month, 1, year))
    to_date = from_date + pd.DateOffset(months=1)
    offset_date = from_date - pd.DateOffset(months=trans_offset)
    
    # Get view: WireTrans
    view_wiretrans = WireTrans[(WireTrans.trans_date > offset_date) & 
                               (WireTrans.trans_date < to_date)]
    # Get view: CheqTrans
    # view_cheqtrans = CheqTrans[(CheqTrans.trans_date > offset_date) & 
    #                            (CheqTrans.trans_date < to_date)]    

    # Get view: CustInfo
    view_customer = CustInfo[CustInfo.open_date < to_date]
    
    # Attach label onto CustInfo
    target_list = SARCase[(SARCase.Status_SAR == 4) & 
                          (SARCase.created_date > from_date) & 
                          (SARCase.created_date < to_date)]['customerno'].unique()
    # print ('# of SAR customers: {}'.format(len(target_list)))
    view_customer['label'] = view_customer.apply(lambda x: 1 if x['customerno'] in target_list else 0, axis=1)
    view_customer  = view_customer.set_index('customerno')
    view_wiretrans = view_wiretrans.set_index('customerno')
    # view_cheqtrans = view_cheqtrans.set_index('customerno')
    return view_wiretrans, view_customer

## Large transaction times in one month

In [19]:
from_date_ym = DateYM(2017, 8)
to_date_ym = DateYM(2019, 6)
trans_out = 500000
trans_in  = 1000000

list_date_seq = list_date_tuples(from_date_ym, to_date_ym)
projectors_shape = dict()
for i, date_ym in zip(range(len(list_date_seq)), list_date_seq):
    print("Processing the data in {}".format(date_ym))
    
    view_wiretrans, view_customer = fetch_data_by_month(date_ym, trans_offset=1)
    view_wiretrans['edge_group'] = view_wiretrans.apply(lambda x: "{}_{}".format(x.org_cust, x.bnf_cust), axis=1)
    
    ## Transact out counts in one month is large than 500,000
    Large_TransOut_Count = view_wiretrans[view_wiretrans.WIRE_AMTOT > trans_out].edge_group.value_counts().rename('Large_TransOut_Count')
    
    ##　WireTrans Out counts in one month
    TransOut_Count = view_wiretrans[view_wiretrans.WIRE_AMTOT > 0].edge_group.value_counts().rename('TransOut_Count')
    
    ##　Transact in total amount per month is large than 1,000,000
    Total_Large_TransIn = view_wiretrans[view_wiretrans.WIRE_AMTIN > trans_in].groupby('edge_group').sum()['WIRE_AMTIN'].rename('Total_Large_TransIn')
    
    ## Total WireTrans Out in one month
    Total_WireTrans = view_wiretrans[view_wiretrans.WIRE_AMTOT > 0].groupby('edge_group').sum()['WIRE_AMTOT'].rename('Total_WireTrans')
    
    ## Total WireTrans Out times in one month
    Total_WireTrans_Times = view_wiretrans[view_wiretrans.WIRE_AMTOT > 0].groupby('edge_group').size().rename('Total_WireTrans_Times')
    
    ## Average WireTrans Out in one month
    Average_WireTrans = (Total_WireTrans/Total_WireTrans_Times).rename('Average_WireTrans')
    
    
    
    ## WireTrans in more than 8,000, and total amount is more than 60,000,000
    WireTransIn_8000 = view_wiretrans[view_wiretrans.WIRE_AMTIN > 8000].groupby('edge_group').sum()
    WireTransIn_8000 = WireTransIn_8000['WIRE_AMTIN'][WireTransIn_8000['WIRE_AMTIN']>60000000].to_frame()
    WireTransIn_8000['WireTransIn_8000'] = 1
    WireTransIn_8000 = WireTransIn_8000['WireTransIn_8000']
    
    ## 9 months
    view_wiretrans, view_customer = fetch_data_by_month(date_ym, trans_offset=9)
    view_wiretrans['edge_group'] = view_wiretrans.apply(lambda x: "{}_{}".format(x.org_cust, x.bnf_cust), axis=1)
    ## WireTrans more than 8,000 in nine months
    WireTrans_Out_9mon = view_wiretrans[view_wiretrans.WIRE_AMTOT > 8000].groupby('edge_group').sum()['WIRE_AMTOT'].rename('WireTrans_Out_9mon')
    
    ## Output
    output = pd.concat([Large_TransOut_Count, 
                        TransOut_Count, 
                        Total_Large_TransIn, 
                        Total_WireTrans, 
                        Total_WireTrans_Times, 
                        Average_WireTrans,
                        WireTransIn_8000,
                        WireTrans_Out_9mon], axis=1).fillna(0)
    output = pd.concat([pd.DataFrame([idx.split('_') for idx in output.index], index=output.index), output], axis=1)
    output.to_csv("./Edge_Attribute/Edge_attribue_{}-{}.csv".format(date_ym[0], date_ym[1]), index=False)

Processing the data in (2017, 8)
Processing the data in (2017, 9)
Processing the data in (2017, 10)
Processing the data in (2017, 11)
Processing the data in (2017, 12)
Processing the data in (2018, 1)
Processing the data in (2018, 2)
Processing the data in (2018, 3)
Processing the data in (2018, 4)
Processing the data in (2018, 5)
Processing the data in (2018, 6)
Processing the data in (2018, 7)
Processing the data in (2018, 8)
Processing the data in (2018, 9)
Processing the data in (2018, 10)
Processing the data in (2018, 11)
Processing the data in (2018, 12)
Processing the data in (2019, 1)
Processing the data in (2019, 2)
Processing the data in (2019, 3)
Processing the data in (2019, 4)
Processing the data in (2019, 5)
Processing the data in (2019, 6)
